In [1]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [18]'.</span>

In [2]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [17]'.</span>

In [3]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [16]'.</span>

In [4]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [15]'.</span>

In [5]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [14]'.</span>

In [6]:
%matplotlib inline

In [7]:
import cdsapi
import os
import urllib
import pathlib

In [8]:
HOME = pathlib.Path.home()

In [9]:
provider = 'CDS'

In [10]:
# list of centres 'ecmwf', 'ukmo', 'meteo_france', 'dwd', 'cmcc'

In [11]:
# Parameters
GCM = "CMCC"
var_name = "T2M"
year = 2019
month = 11


In [12]:
opath = HOME  / 'drives' / 'auck_projects' / 'END19101' / 'Working' / 'data' / 'GCMs' / 'downloads' / 'forecasts' / provider / GCM / var_name 

In [13]:
opath

PosixPath('/home/nicolasf/drives/auck_projects/END19101/Working/data/GCMs/downloads/forecasts/CDS/CMCC/T2M')

In [14]:
if not opath.exists(): 
    opath.mkdir(parents=True)

In [15]:
# dictionnary mapping the variable name (to be used in the downloaded filename) to the variable name in the CDS datasets
var_dict = {}
var_dict['T2M'] = '2m_temperature'
var_dict['PRECIP'] = 'total_precipitation'
var_dict['SST'] = 'sea_surface_temperature'
var_dict['Z850'] = 'geopotential'
var_dict['Z500'] = 'geopotential'

In [16]:
def fetch_forecast(GCM, var_name, year, month, opath=None):

    centre = GCM.lower()
    
    month = str(month).zfill(2)
    year = str(year)
    
    if "Z" in var_name:
        
        level = var_name[-3:]

        c = cdsapi.Client()
        try:
            data = c.retrieve(
            'seasonal-monthly-pressure-levels',
            {
                'originating_centre':centre,
                'variable':var_dict[var_name],
                'pressure_level':level,
                'product_type':'monthly_mean',
                'year':year,
                'month':month,
                'leadtime_month':[
                    '2','3','4',
                    '5','6'
                ],
                'format':'grib'
            },
            f'{str(opath)}/{GCM}_{var_name}_{year}_{month}.grib')
        except:
            print(f"unable to download {var_name} for {year}-{month} for GCM {GCM}")
            data = None

    else:

        c = cdsapi.Client()
        try:
            data = c.retrieve(
                'seasonal-monthly-single-levels',
                {
                    'originating_centre':centre,
                    'variable':var_dict[var_name],
                    'product_type':'monthly_mean',
                    'year':str(year),
                    'month':str(month),
                    'leadtime_month':[
                        '2','3','4',
                        '5','6'
                    ],
                    'format':'grib'
                },
                f'{str(opath)}/{GCM}_{var_name}_{year}_{month}.grib')
        except:
            print(f"unable to download {var_name} for {year}-{month} for GCM {GCM}")
            data = None
            
    return data 

In [ ]:
data = fetch_forecast(GCM, var_name, year, month, opath=opath)

In [ ]:
filename = pathlib.Path(opath, f'{str(opath)}/{GCM}_{var_name}_{str(year)}_{str(month).zfill(2)}.grib') 

In [ ]:
if filename.exists():
    print(f"{filename} already downloaded")
    pass
else:
    print(f"trying to download {GCM} monthly for year {year} and month {month}")
    data = fetch_forecast(GCM, var_name, year, month, opath=opath)
    if filename.exists(): 
        data.delete()
        print(f"cleaning up data on the CDS at {data.location}")
    else: 
        try:
            data.download(target=filename)
        except: 
            print(f"\n\n\n\nunable to download {filename} from {data.location}\n\n\n")